<a href="https://colab.research.google.com/github/AbhishekRP2002/llm-learning-cookbook/blob/main/langchain/Query_Rewrite_RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-openai
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install chromadb
# !pip install ragas
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [ ]:
import os
import openai
from openai import OpenAI
import time
import json
import tiktoken
from getpass import getpass as gp
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, OpenAI
from langchain_community.vectorstores import Chroma as ch, DeepLake
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import RetrievalQAWithSourcesChain , RetrievalQA, create_retrieval_chain



In [ ]:
api_key = gp("Enter your API Key : ")

Enter your API Key : ··········


In [ ]:
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
loader = WebBaseLoader("https://paulgraham.com/smart.html")

data = loader.load()
chunk_size = 1000
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)


In [ ]:
#split the document as per chunksize and chunkoverlap
splitdocument = text_splitter.split_documents(data)

#create embeddings
embeddings = OpenAIEmbeddings()

# Create a vector database using Chroma
vectDB = ch.from_documents(splitdocument,embeddings)
retriever = vectDB.as_retriever(return_source_document=True)

In [ ]:
model = ChatOpenAI()


In [ ]:
prompt_template = """
You are an helpful AI assistant capable of understanding complex questions by decomposing it into smaller directives/ questions.In order to get the response of a complex question, follow the following steps :
- Recursively break-down the post into smaller questions/directives
- For each atomic question/directive:
  - Select the most relevant information from the context in light of the conversation history
  - Generate a draft response using the selected information, whose brevity/detail are tailored to the poster’s expertise
  - Remove duplicate content from the draft response
- Generate your final response after adjusting it to increase accuracy and relevance
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)


In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("What defines intelligence, and why has society historically placed such a high value on it?")

'Intelligence is often defined as the ability to acquire and apply knowledge and skills. Society has historically placed a high value on intelligence for several reasons. Firstly, intelligence is often associated with problem-solving abilities and innovation, which are highly valued in many fields. Secondly, intelligence is often linked to success and achievement, and individuals with higher intelligence may have better opportunities for career advancement and social status. Additionally, intelligence is often seen as a measure of potential and capability, and individuals with higher intelligence may be perceived as having greater potential for success and contributions to society. It is important to note that intelligence is not the sole determinant of success or value, and there are other qualities and factors that contribute to individual and societal well-being.'

In [ ]:
retrievalQA = RetrievalQA.from_llm(llm=OpenAI(), retriever=retriever, return_source_documents=True,verbose =True)

In [ ]:
retrievalQA.invoke("What defines intelligence, and why has society historically placed such a high value on it?")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What defines intelligence, and why has society historically placed such a high value on it?',
 'result': ' Intelligence is defined as the ability to acquire and apply knowledge and skills. Society has historically placed a high value on intelligence because it is seen as a means to success and a way to stand out in a competitive society. Additionally, intelligence has been equated with being "smart" or "gifted," leading to a cultural emphasis on being intelligent as a desirable trait.',
 'source_documents': [Document(page_content="of new ideas, and you're constantly being judged by it. Whereas\neven the kids who will ultimately discover new things aren't usually\ndiscovering them yet. For kids that way inclined, intelligence is\nthe only game in town.There are more subtle reasons too, which persist long into adulthood.\nIntelligence wins in conversation, and thus becomes the basis of\nthe dominance hierarchy.\n[1]\nPlus having new ideas is such a new\nthing historically, and